In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)

x1 = np.random.uniform(0, 10, 1000)
x2 = np.random.uniform(0, 10, 1000)

# luật gốc tự sinh
y = np.where((x1 > 5) & (x2 < 3), 1, 0)
df = pd.DataFrame({'x1': x1, 'x2': x2, 'y': y})
print(df.head())


         x1        x2  y
0  3.745401  1.851329  0
1  9.507143  5.419009  0
2  7.319939  8.729458  0
3  5.986585  7.322249  0
4  1.560186  8.065611  0


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from util import *

def prepare_dataset(df):
    """
    Chuẩn bị dataset với các bước:
    - Chuẩn hóa cột liên tục
    - Encode cột phân loại (nếu có)
    - Tách X và y
    """
    # Lấy danh sách các cột
    columns = df.columns.tolist()

    # Tên cột target
    class_name = 'y'

    # Kiểm tra các giá trị duy nhất của cột class
    possible_outcomes = list(df[class_name].unique())

    type_features, features_type = recognize_features_type(df, class_name)
    print(type_features)
    discrete = ['y']
    discrete, continuous = set_discrete_continuous(columns, type_features, class_name, discrete=discrete,
                                                   continuous=None)
    print(discrete, continuous)

    columns_tmp = list(columns)
    columns_tmp.remove(class_name)
    idx_features = {i: col for i, col in enumerate(columns_tmp)}

    # Dataset Preparation for Scikit Alorithms
    df_le, label_encoder = label_encode(df, discrete)
    X = df_le.loc[:, df_le.columns != class_name].values
    y = df_le[class_name].values

    # Chuẩn bị dataset trả về
    dataset = {
        'df': df,  # DataFrame đã chuẩn bị
        'columns': columns,  # Danh sách cột
        'class_name': class_name,  # Tên cột target
        'possible_outcomes': possible_outcomes,  # Giá trị duy nhất của y
        'type_features': type_features,  # Loại của từng cột
        'features_type': features_type,
        'discrete': discrete,  # Cột phân loại (nếu có)
        'continuous': continuous,  # Cột liên tục
        'label_encoder': label_encoder,   
        'idx_features': idx_features,  
        'X': X,  # Dữ liệu đầu vào
        'y': y  # Nhãn đầu ra
    }

    return dataset

In [3]:
print("Cột y trước khi mã hóa:", df['y'].unique())
dataset = prepare_dataset(df)
print("Cột y sau khi mã hóa:", dataset['y'])


Cột y trước khi mã hóa: [0 1]
{'integer': ['y'], 'double': ['x1', 'x2'], 'string': []}
['y'] ['x1', 'x2']
Cột y sau khi mã hóa: [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0
 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1
 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1
 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0
 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


# Chia tập train/test
X, y = dataset['X'], dataset['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [5]:
import lore
from prepare_dataset import *
from neighbor_generator import *

path_data = 'datasets/'
idx_record2explain = 0
X2E = X_test

explanation, infos = lore.explain(idx_record2explain, X2E, dataset, model,
                                      ng_function=genetic_neighborhood,
                                      discrete_use_probabilities=True,
                                      continuous_function_estimation=False,
                                      returns_infos=True,
                                      path=path_data, sep=';', log=False)

x = [3.80890857 9.34436027]
Giá trị của columns: ['x1', 'x2', 'y']
yX sau ghép: [[3.80890857 9.34436027 0.        ]
 [8.16599439 1.94622512 1.        ]
 [4.62679786 8.27537896 0.        ]
 [3.56096726 6.00594333 0.        ]
 [9.50061967 1.11919619 1.        ]]
Giá trị của cột x1 sau ghép: [0.05061584 0.05522117 0.14393489 0.15456617 0.18221826 0.18390677
 0.20584494 0.23271936 0.24315966 0.25350743 0.26532204 0.31183135
 0.33532435 0.40775142 0.40868616 0.4544638  0.51425813 0.56375497
 0.65708639 0.72763006 0.81348781 0.81759032 0.85347465 0.85508531
 0.86920288 0.91582073 0.97176494 1.00778001 1.04108582 1.17067016
 1.17751083 1.19762132 1.20635871 1.27060513 1.38827173 1.40924225
 1.51902694 1.55240246 1.56437043 1.60808051 1.69492747 1.73373595
 1.81233066 1.81435435 1.8192813  1.86567024 1.87043749 1.8812116
 2.09349323 2.14940379 2.21972162 2.24269309 2.28642806 2.37249087
 2.37637544 2.50242898 2.57941628 2.58779982 2.70832251 2.73407072
 2.76877648 2.78590339 2.79979094 2.83775

/Users/admin/Documents/Screenshots/LORE/.env/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/Users/admin/Documents/Screenshots/LORE/.env/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


OSError: [Errno 8] Exec format error: 'yadt/dTcmd'

In [ ]:
print('r = %s --> %s' % (explanation[0][1], explanation[0][0]))